# Data Analyzer - Homomorphic Computation

This notebook performs calculations on encrypted data without access to the secret key.

In [1]:
import tenseal as ts
import utils
import json

------------------------------------------

## Load Public Context

Load the public encryption context that allows computations but not decryption.

In [2]:
# Load CKKS public context (for radiation data)
ckks_public_context_bytes = utils.read_data("keys/ckks_public.txt")
ckks_public_context = ts.context_from(ckks_public_context_bytes)

# Load BFV public context (for enemy count data)
bfv_public_context_bytes = utils.read_data("keys/bfv_public.txt")
bfv_public_context = ts.context_from(bfv_public_context_bytes)

------------------------------------------

## Load Encrypted Radiation Data

In [3]:
# Load encrypted radiation data (CKKS)
encrypted_radiation_bytes = utils.read_data("outputs/radiation_encrypted.txt")
encrypted_radiation_vector = ts.lazy_ckks_vector_from(encrypted_radiation_bytes)
encrypted_radiation_vector.link_context(ckks_public_context)

# Load encrypted enemy count data (BFV)
encrypted_enemy_bytes = utils.read_data("outputs/enemy_encrypted.txt")
encrypted_enemy_vector = ts.lazy_bfv_vector_from(encrypted_enemy_bytes)
encrypted_enemy_vector.link_context(bfv_public_context)

------------------------------------------

## Load Encrypted Calculation Parameters

Load pre-encrypted weights, divisor, and thresholds. The Analyzer never sees the plaintext values.

In [4]:
# Load encrypted weights (Analyzer cannot see 0.7 and 0.3)
encrypted_weight_radiation_bytes = utils.read_data("outputs/weight_radiation_encrypted.txt")
encrypted_weight_radiation = ts.lazy_ckks_vector_from(encrypted_weight_radiation_bytes)
encrypted_weight_radiation.link_context(ckks_public_context)

encrypted_weight_enemy_bytes = utils.read_data("outputs/weight_enemy_encrypted.txt")
encrypted_weight_enemy = ts.lazy_ckks_vector_from(encrypted_weight_enemy_bytes)
encrypted_weight_enemy.link_context(ckks_public_context)

# Load encrypted divisor (Analyzer cannot see number of measurements)
encrypted_divisor_bytes = utils.read_data("outputs/divisor_encrypted.txt")
encrypted_divisor = ts.lazy_ckks_vector_from(encrypted_divisor_bytes)
encrypted_divisor.link_context(ckks_public_context)

# Load encrypted thresholds (Analyzer cannot see 50.0 and 10.0)
encrypted_threshold_radiation_bytes = utils.read_data("outputs/threshold_radiation_encrypted.txt")
threshold_radiation = ts.lazy_ckks_vector_from(encrypted_threshold_radiation_bytes)
threshold_radiation.link_context(ckks_public_context)

encrypted_threshold_enemies_bytes = utils.read_data("outputs/threshold_enemies_encrypted.txt")
threshold_enemies = ts.lazy_ckks_vector_from(encrypted_threshold_enemies_bytes)
threshold_enemies.link_context(ckks_public_context)

# Load encrypted constant (Analyzer cannot see 100.0)
encrypted_safe_route_constant_bytes = utils.read_data("outputs/safe_route_constant_encrypted.txt")
safe_route_constant = ts.lazy_ckks_vector_from(encrypted_safe_route_constant_bytes)
safe_route_constant.link_context(ckks_public_context)

print("✅ Loaded all encrypted parameters")
print("   → Calculation logic (weights, divisor, thresholds) remains hidden")

✅ Loaded all encrypted parameters
   → Calculation logic (weights, divisor, thresholds) remains hidden


------------------------------------------

## Step 1: BFV Workflow (Enemy Count Data - Integer Operations)

In [5]:
# BFV: Sum all encrypted enemy counts (exact integer addition)
encrypted_enemy_sum = encrypted_enemy_vector.sum()

print("✅ BFV Workflow Complete: Computed encrypted enemy sum (total across all drones)")
print(" → Only integer operations, no weights or thresholds needed")

✅ BFV Workflow Complete: Computed encrypted enemy sum (total across all drones)
 → Only integer operations, no weights or thresholds needed


------------------------------------------

## Step 2: CKKS Workflow (Radiation Data - Float Operations)

In [6]:
# CKKS supports floating-point operations: multiplication, division, weighted sums
# Using pre-encrypted parameters (weights, divisor, thresholds, constants)
# Analyzer does NOT see plaintext values

# Step 2.1: Compute Radiation Average
encrypted_radiation_sum = encrypted_radiation_vector.sum()
encrypted_radiation_average = encrypted_radiation_sum * encrypted_divisor
print("✅ Computed encrypted radiation average (using hidden divisor 1/n)")

# Step 2.2: Weighted Radiation Score
encrypted_radiation_score = encrypted_radiation_average * encrypted_weight_radiation
print("✅ Computed weighted radiation score (using hidden weight 0.7)")

# Step 2.3: High-Risk Detection
encrypted_radiation_excess = encrypted_radiation_average - threshold_radiation
print("✅ Computed radiation excess above threshold (using hidden threshold 50.0 mSv)")

# Step 2.4: Safe Route Score
encrypted_safe_route_score = safe_route_constant - encrypted_radiation_score
print("✅ Computed safe route score (using hidden constant 100.0)")

print("\n✅ CKKS Workflow Complete: All float operations on encrypted radiation data")

✅ Computed encrypted radiation average (using hidden divisor 1/n)
✅ Computed weighted radiation score (using hidden weight 0.7)
✅ Computed radiation excess above threshold (using hidden threshold 50.0 mSv)
✅ Computed safe route score (using hidden constant 100.0)

✅ CKKS Workflow Complete: All float operations on encrypted radiation data


------------------------------------------

## Save Results

Save all encrypted computation results for the Data Holder to decrypt and verify.

In [7]:
# Save CKKS results (radiation analysis)
serialized_encrypted_average = encrypted_radiation_average.serialize()
utils.write_data("outputs/radiation_average_encrypted.txt", serialized_encrypted_average)

serialized_radiation_score = encrypted_radiation_score.serialize()
utils.write_data("outputs/radiation_score_encrypted.txt", serialized_radiation_score)

serialized_safe_route = encrypted_safe_route_score.serialize()
utils.write_data("outputs/safe_route_score_encrypted.txt", serialized_safe_route)

# Save BFV results (enemy count analysis)
serialized_encrypted_sum = encrypted_enemy_sum.serialize()
utils.write_data("outputs/enemy_sum_encrypted.txt", serialized_encrypted_sum)

print("✅ CKKS results saved:")
print("   - radiation_average_encrypted.txt")
print("   - radiation_score_encrypted.txt")
print("   - safe_route_score_encrypted.txt")
print("\n✅ BFV results saved:")
print("   - enemy_sum_encrypted.txt")
print("\n✅ Data Holder can now decrypt all results")

✅ CKKS results saved:
   - radiation_average_encrypted.txt
   - radiation_score_encrypted.txt
   - safe_route_score_encrypted.txt

✅ BFV results saved:
   - enemy_sum_encrypted.txt

✅ Data Holder can now decrypt all results
